# Chapter 16: Visualizing Data

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.8.2

## NYC Bike Trips

In [ ]:
import polars as pl

trips = pl.read_parquet("data/biketrips/*.parquet")

print(trips[:,:4])
print(trips[:,4:7])
print(trips[:,7:11])
print(trips[:,11:])

In [ ]:
# Sampling is needed to keep the filesize of this notebook relatively low.
# In the book we use the full DataFrame.

trips = trips.sample(fraction=0.1)

In [ ]:
trips = trips.filter(
    ~((pl.col("station_start") == pl.col("station_end")) &
    (pl.col("duration").dt.total_seconds() < 5*60))
)
trips.height

## Built-in Plotting with hvPlot

### A First Plot

In [ ]:
trips_speed = (
    trips
    .filter(pl.col("station_start") == "W 21 St & 6 Ave")
    .select(  
        pl.col("distance"),
        pl.col("duration").dt.total_seconds() / 3600,  
        pl.col("bike_type")
    )
)
trips_speed

In [ ]:
trips_speed.plot.scatter(x="distance", y="duration", color="bike_type",  
                         xlabel="distance (km)", ylabel="duration (h)",  
                         ylim=(0, 2))  

### Methods in the Plot Namespace

In [ ]:
trips.plot.<TAB>

### Getting Help for a Method

In [ ]:
import hvplot
hvplot.help('scatter', generic=False, style=False)

### Pandas as Backup

In [ ]:
trips_per_day_hour = (
    trips
    .sort("datetime_start")
    .group_by_dynamic("datetime_start", every="1h")
    .agg(pl.len())
)

trips_per_day_hour.plot.heatmap(x='datetime_start.hour',
                                y='datetime_start.day',
                                C='len', cmap='reds')

In [ ]:
import hvplot.pandas
trips_per_day_hour.to_pandas().hvplot.heatmap(x='datetime_start.hour',
                                              y='datetime_start.day',
                                              C='len', cmap='reds')

### Manual Transformations

In [ ]:
trips_type_counts = trips.group_by("rider_type", "bike_type").len()
trips_type_counts

In [ ]:
trips_type_counts.plot.bar(x="rider_type", y="len", by="bike_type",
                           ylabel="count", stacked=True,
                           color=["orange", "green"])

### Changing the Plotting Backend

In [ ]:
import hvplot
hvplot.extension("matplotlib")

In [ ]:
trips_type_counts.plot.bar(x="rider_type", y="len", by="bike_type",
                           ylabel="count", stacked=True,
                           color=["orange", "green"])

In [ ]:
hvplot.extension("bokeh")

### Plotting Points on a Map

In [ ]:
trips.plot.scatter(x='lon_start', y='lat_start', color='borough_start',
                   width=600, height=600)

In [ ]:
trips.plot.points(x="lon_start", y="lat_start",
                  datashade=True, geo=True,
                  tiles="CartoLight",
                  width=800, height=600)

### Composing Plots

In [ ]:
trips_hour_num_speed = (
    trips
    .sort("datetime_start")
    .group_by_dynamic("datetime_start", every="1h")
    .agg(num_trips=pl.len(),
         speed=(pl.col("distance") / (pl.col("duration").dt.total_seconds() / 3600)).median())
    .filter(pl.col("datetime_start") > pl.date(2024, 3 , 26))
)

trips_hour_num_speed

In [ ]:
(
    trips_hour_num_speed.plot.line(x="datetime_start", y="num_trips") +
    trips_hour_num_speed.plot.line(x="datetime_start", y="speed")
).cols(1)   

In [ ]:
(
    trips_hour_num_speed.plot.line(x="datetime_start", y="num_trips") *
    trips_hour_num_speed
        .filter(pl.col("num_trips") > 9000)
        .plot.scatter(x="datetime_start", y="num_trips", c="red", s=50)
)

### Adding Interactive Widgets

In [ ]:
trips_per_hour = (
    trips
    .sort("datetime_start")
    .group_by_dynamic("datetime_start", group_by="borough_start", every="1h")
    .agg(pl.len())
    .with_columns(date=pl.col("datetime_start").dt.date())
)
trips_per_hour

In [ ]:
trips_per_hour.plot.line(x="datetime_start", by="borough_start",
                         groupby="date", widget_location='left_top')

### Common Customizations

In [ ]:
busiest_stations = (
    trips
    .group_by(station="station_start").agg(
        num_trips=pl.len(),
    )
    .sort("num_trips", descending=True)
    .head(20)
)

busiest_stations

In [ ]:
fig = busiest_stations.plot.bar(x="station", y="num_trips", color="num_trips",
                                cmap="viridis",
                                fontscale=1.2,
                                grid=True,
                                logx=False, logy=True,
                                rot=45,
                                title="Busiest Citi Bike Stations",
                                width=800, height=400,
                                xlabel="", ylabel="Number of trips",
                                xlim=None, ylim=(4000, None))

fig

In [ ]:
def bokeh_hook(plot, element):
    plot.handles["yaxis"].major_label_text_color = "blue"
    plot.handles["plot"].title.align = "right"

fig.opts(invert_axes=True, invert_yaxis=True, hooks=[bokeh_hook])

## Alternative Packages

### Plotnine

In [ ]:
from plotnine import *

In [ ]:
trips_speed = (
    trips.group_by("neighborhood_start", "neighborhood_end").agg(
        pl.col("duration").dt.total_seconds().median() / 3600,
        pl.col("distance").median(),
        pl.col("borough_start").first(),
        pl.col("borough_end").first(),
        pl.len(),
    ).filter(
        (pl.col("len") > 30) &
        (pl.col("distance") > 0.2) &
        (pl.col("neighborhood_start") != pl.col("neighborhood_end")),
    ).with_columns(
        speed=pl.col("distance") / pl.col("duration")
    ).sort("borough_start")
)
trips_speed

In [ ]:
(
    ggplot(data=trips_speed
        .filter(pl.col("borough_start") == pl.col("borough_end")),
        mapping=aes(x="distance", y="duration", color="borough_end")) +
    geom_point(size=0.25, alpha=0.5) +
    geom_smooth(method="lowess", size=2, se=False, alpha=0.8) +
    scale_color_brewer(type="qualitative", palette="Set1") +
    labs(title="Trip distance and duration within each borough",
         x="Distance (km)", y="Duration (m)", color="Borough") +
    theme_linedraw() +
    theme(figure_size=(8, 6))
)

In [ ]:
(
    ggplot(data=trips_speed
        .filter(pl.col("borough_start") != pl.col("borough_end"))
        .with_columns(
            ("From " + pl.col("borough_start")).alias("borough_start")),
        mapping=aes(x="distance", y="duration", color="borough_end")) +
    geom_point(size=0.25, alpha=0.5) +
    geom_smooth(method="lowess", size=2, se=False, alpha=0.8) +
    scale_color_brewer(type="qualitative", palette="Set1") +
    facet_wrap("borough_start") +
    labs(title="Trip distance and duration cross borough",
         x="Distance (km)", y="Duration (m)", color="To Borough") +
    theme_linedraw() +
    theme(figure_size=(8, 6))
)

### Great Tables

In [ ]:
busiest_stations = (
    trips
    .group_by(   
        station=pl.col("station_start"),
        date=pl.col("datetime_start").dt.date()
    )
    .agg(
        borough=pl.col("borough_start").first(),
        neighborhood=pl.col("neighborhood_start").first(),
        num_rides=pl.len(),
        percent_member=(pl.col("rider_type") == "member").mean(),
        percent_electric=(pl.col("bike_type") == "electric").mean()
    )
    .sort("date")
    .group_by("station")
    .agg(
        pl.col(pl.String).first(),
        pl.col(pl.NUMERIC_DTYPES).mean(),
        pl.col("num_rides").alias("rides_per_day")  
    )
    .sort("num_rides", descending=True)
    .group_by("borough", maintain_order=True).head(3)
)
busiest_stations

In [ ]:
import polars.selectors as cs
from great_tables import GT, style, md

(
    GT(busiest_stations, rowname_col="station", groupname_col="borough") 
    .cols_label(       
        neighborhood="Neighborhood",
        num_rides="Mean Daily Rides",
        percent_member="Members",
        percent_electric="E-Bikes",
        rides_per_day="Rides Per Day",
    )
    .tab_header(
        title="Busiest Bike Stations in NYC",
        subtitle="In March 2024, Per Borough"
    )
    .tab_stubhead(label="Station")
    .fmt_number(columns="num_rides", decimals=1)
    .fmt_percent(columns=cs.starts_with("percent_"), decimals=0)  
    .fmt_nanoplot(columns="rides_per_day", reference_line="mean")
    .data_color(columns="num_rides", palette="Blues")
    .tab_options(row_group_font_weight="bold")
    .tab_source_note(source_note=md(
        "Source: [NYC Citi Bike](https://citibikenyc.com/system-data)"
    ))
)

## Takeaways